In [1]:
#импортируем необходимые библиотеки
import cv2
import mediapipe as mp #face detector
import math
import numpy as np
import torch
from PIL import Image
from torchvision import transforms

#### Sub functions

In [2]:
#функция для предобработки изображений перед передачей их на модель торча
def pth_processing(fp):
    #класс используется для выполнения предварительной обработки входного массива данных (тензора)
    class PreprocessInput(torch.nn.Module):
        def init(self):
            #вызывается инициализация родительского класса с помощью `super()`, чтобы убедиться, что базовые настройки корректны
            super(PreprocessInput, self).init()

        #метод определяет, как данные будут обрабатываться, когда они проходят через модуль
        def forward(self, x):
            #преобразуем входной тензор `x` в тип данных `torch.float32`
            x = x.to(torch.float32)
            #обращаем порядок всех каналов (например, если это изображение с тремя цветными каналами RGB, то порядок будет изменен с RGB на BGR)
            x = torch.flip(x, dims=(0,))
            #проводим нормализацию каналов RGB, вычитая специфические значения средних значений для каждого канала для того, 
            #чтобы улучшить качество входных данных перед подачей в модель
            x[0, :, :] -= 91.4953
            x[1, :, :] -= 103.8827
            x[2, :, :] -= 131.0912
            return x

    #обработки изображения, чтобы подготовить его для модели
    def get_img_torch(img):
        
        #создаем последовательность преобразований (pipeline), которая сначала преобразует изображение в тензор, 
        #а затем применяет к нему модуль `PreprocessInput`
        ttransform = transforms.Compose([
            transforms.PILToTensor(),
            PreprocessInput()
        ])
        img = img.resize((224, 224), Image.Resampling.NEAREST)
        img = ttransform(img)
        #добавляем новую ось к тензору, чтобы подготовить его к обработке моделью 
        #(т.е. делаем его батчем с размерностью `(1, C, H, W)`) и перемещаем его на CPU.
        img = torch.unsqueeze(img, 0).to('cpu')
        return img
    return get_img_torch(fp)

#преобразовываем нормализованные координаты в пиксельные
def norm_coordinates(normalized_x, normalized_y, image_width, image_height):
    
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    
    return x_px, y_px

#функция нужна для вычисления ограничивающего прямоугольника (bounding box) на основе заданных координат ключевых точек (landmarks)
def get_box(fl, w, h):
    #пустой словарь `idx_to_coors`, который будет использоваться для сопоставления индексов ключевых точек с их пиксельными координатами
    idx_to_coors = {}
    #цикл по ключевым точкам
    for idx, landmark in enumerate(fl.landmark):
        landmark_px = norm_coordinates(landmark.x, landmark.y, w, h)

        if landmark_px:
            idx_to_coors[idx] = landmark_px

    #вычисление ограничивающего прямоугольника
    x_min = np.min(np.asarray(list(idx_to_coors.values()))[:,0])
    y_min = np.min(np.asarray(list(idx_to_coors.values()))[:,1])
    endX = np.max(np.asarray(list(idx_to_coors.values()))[:,0])
    endY = np.max(np.asarray(list(idx_to_coors.values()))[:,1])

    #ограничение координат
    (startX, startY) = (max(0, x_min), max(0, y_min))
    (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
    
    return startX, startY, endX, endY

#используется для отображения результатов предсказаний на изображении
def display_EMO_PRED(img, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255), line_width=2, ):
    #задание параметров для рисования
    lw = line_width or max(round(sum(img.shape) / 2 * 0.003), 2)
    text2_color = (255, 0, 255)
    #определение точек для прямоугольника
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    #рисование ограничивающего прямоугольника
    cv2.rectangle(img, p1, p2, text2_color, thickness=lw, lineType=cv2.LINE_AA)
    #определение параметров текста
    font = cv2.FONT_HERSHEY_SIMPLEX
    tf = max(lw - 1, 1)
    text_fond = (0, 0, 0)
    text_width_2, text_height_2 = cv2.getTextSize(label, font, lw / 3, tf)
    text_width_2 = text_width_2[0] + round(((p2[0] - p1[0]) * 10) / 360)
    center_face = p1[0] + round((p2[0] - p1[0]) / 2)

    #добавление текста на изображение
    cv2.putText(img, label,
                (center_face - round(text_width_2 / 2), p1[1] - round(((p2[0] - p1[0]) * 20) / 360)), font,
                lw / 3, text2_color, thickness=tf, lineType=cv2.LINE_AA)
    return img

#### Testing models by webcam

In [3]:
#инициализируем объекты mediapipe для рисования лицевой сетки и контуров
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#создаем объект, определяющий стиль рисования для контуров лица
my_drawing_specs = mp_drawing.DrawingSpec(color = (0, 255, 0), thickness = 1)

#задаем переменную, которая хранит имя модели для удобства
name = '0_66_49_wo_gl'

#с помощью торча методом компиляции в моменте загружаем обученную модель на CPU (вместо 0 вставится name)
pth_model = torch.jit.load('models_EmoAffectnet/torchscript_model_{0}.pth'.format(name)).to('cpu')
#переключение модели в режим валидации (оценки)
pth_model.eval()

#создаем словарь, в котором определяем перечень эмоций
DICT_EMO = {0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger'}

#открываем первую доступную камеру
cap = cv2.VideoCapture(0)
#получаем ширину и высоту кадра
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#инциализируем модуль Face Mesh
mp_face_mesh = mp.solutions.face_mesh
#параметры для стабилизации эмоций
emotion_history = []  #история предсказанных эмоций для усреднения и предотвращения резких смен эмоций
max_history_length = 10  #сколько последних предсказаний сохранять для сглаживания
dominant_emotion = "Neutral"  #текущая доминирующая эмоция
previous_emotion = "Neutral"  #предыдущая эмоция для плавной смены
#количество кадров с новой эмоцией для подтверждения смены эмоции
emotion_threshold = 5
#минимальный уровень уверенности для принятия эмоции
confidence_threshold = 0.9  #минимальная уверенность (от 0 до 1)
#переменные для отслеживания стабильности эмоций
emotion_change_counter = 0  #счетчик изменения эмоции
current_emotion_streak = 0  #количество кадров с текущей новой эмоцией
last_detected_emotion = "Neutral"  #последняя обнаруженная эмоция
  
with mp_face_mesh.FaceMesh(
max_num_faces=1, #количество одновременно распознанных лиц на изображении
refine_landmarks=True, #уточненные ключевые точки лица для детализированного результата
min_detection_confidence=0.5,  #порог уверенности в том, что найденный объект является лицом
min_tracking_confidence=0.5) as face_mesh: #минимальный уровень уверенности, необходимый для отслеживания лица между кадрами

    #основной цикл обработки кадров
    while cap.isOpened():
        #захватываем кадр с камеры; возвращаются успешность захвата и кадр в виде массива пикселей
        success, img = cap.read()
        #если возникли проблемы с захватом изображения с камеры, закрываем окно
        if not success: break

        #обработка захваченного кадра с помощью модели распознавания лицевой сетки
        results = face_mesh.process(img)

        #если на изображении распознаны лица
        if results.multi_face_landmarks:
            #то для каждого лица
            for fl in results.multi_face_landmarks:

                #рисуется лицевая сетка с использованием стиля FACEMESH_TESSELATION
                mp_drawing.draw_landmarks(
                    #изображение, на которое будут наноситься ключевые точки и их соединения
                    image = img,
                    #ключевые точки, содержащие координаты различных частей лица
                    landmark_list = fl,
                    #соединеняем ключевые точки лица в треугольники
                    connections = mp_face_mesh.FACEMESH_TESSELATION,
                    #ключевые точки рисоваться не будут
                    landmark_drawing_spec = None,
                    #используем предустановленный стиль для рисования лицевой сетки
                    connection_drawing_spec = mp_drawing_styles
                    .get_default_face_mesh_tesselation_style()
                )
                #рисуются контуры лица с использованием стиля рисования my_drawing_specs
                mp_drawing.draw_landmarks(
                    image=img,
                    landmark_list=fl,
                    #какие точки нужно соединить для рисования контуров (по краю лица, глаз, губ и т.д.)
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    #используем кастомный стиль для рисования контуров лица
                    connection_drawing_spec = my_drawing_specs
                    #.get_default_face_mesh_tesselation_style()
                )

                #инициализация начальных координат
                h, w, _ = img.shape
                x_min, y_min, x_max, y_max = w, h, 0, 0

                #для каждой ключевой точки лица вычисляем координаты, определяющие область лица
                for landmark in fl.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y

                #устанавливаем значение отступа (в пикселях), которое будет добавлено к координатам ограничивающего прямоугольника
                padding = 20
                #корректировка координат с добавлением отступа
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)

                #получение ограничивающего прямоугольника для лица
                startX, startY, endX, endY  = get_box(fl, w, h)
                #извлечение области лица из изображения
                cur_face = img[startY:endY, startX: endX]

                #предобработка извлеченной области
                cur_face = pth_processing(Image.fromarray(cur_face))
                #предсказание эмоции с использованием модели
                output = torch.nn.functional.softmax(pth_model(cur_face), dim=1).cpu().detach().numpy()
                
                #получение класса и метки эмоции
                cl = np.argmax(output)
                label = DICT_EMO[cl]

                detected_emotion = label
                #если новая эмоция не совпадает с предыдущей обнаруженной
                if detected_emotion != last_detected_emotion:
                    current_emotion_streak = 0  #сбрасываем счетчик стабильности
                    last_detected_emotion = detected_emotion
                else:
                    current_emotion_streak += 1  #увеличиваем счетчик

                #если эмоция повторяется достаточное количество кадров, она становится доминирующей
                if current_emotion_streak >= emotion_threshold:
                    dominant_emotion = detected_emotion
                    current_emotion_streak = 0

                #плавное изменение цвета текста эмоции для эффекта плавной смены
                if dominant_emotion != previous_emotion:
                    cv2.putText(img, dominant_emotion, (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    previous_emotion = dominant_emotion
                else:
                    cv2.putText(img, dominant_emotion, (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                img = display_EMO_PRED(img, (startX, startY, endX, endY), label, line_width=3)
        
        #отображаем зеркальное изображение в окне с заголовком Webcam
        cv2.imshow('Webcam', cv2.flip(img, 1))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    #освобождаем ресурсы, связанные с камерой
    cap.release()

    #закрываем все окна OpenCV
    cv2.destroyAllWindows()

d:\AISD\OurProject\OurProject\project_av.emotion\Recognize_emotions\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
